In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    # memory_key="chat_history", // 기본값으로 메모리 키가 "history"이기 때문에 모든 chat_history를 이걸로(history) 대체할 수 있음
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

# load_memory_variables는 메모리를 로드시키는 함수
def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

# save_context는 사람과 ai의 메세지인 input과 output을 메모리에 저장
def invoke_chain(question): # question > 사용자 값을 받아옴
    # 랭체인에게 이 함수의 결과와 유저가 프롬프트에 입력한 값을 달라고 함. chain실행
    # langchain은 load_memory를 실행시키고 그걸 history속성값에 넣어준 후에 그 값을 prompt에 넣어주고 
    # 유저에게 받아온 question값도 prompt에 넣어줌
    result = chain.invoke({"question": question}) 
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [2]:
invoke_chain("내 이름은 길동이라고해 반가워!")

content='안녕하세요, 길동님! 만나서 반가워요. 무엇을 도와드릴까요?'


In [3]:
invoke_chain("내 이름이 뭐야?")

content='죄송해요, 제가 당신의 이름을 알 수는 없어요. 그래서 "길동님"이라고 부르는 거예요. 어떤 도움이 필요하신가요?'
